## Query with Memory from txt File 

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

In [52]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import JSONLoader

loader = TextLoader("test_data.txt")
documents = loader.load()
import json

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

Created a chunk of size 1049, which is longer than the specified 1000
Created a chunk of size 1049, which is longer than the specified 1000


In [7]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [8]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory)

In [9]:
query = "From whom is the Document?"
result = qa({"question": query})

In [10]:
result["answer"]

' The document is from Linea Schmidt.'

In [11]:
query2 = "Is there any information about her profession?"
result2 = qa({"question": query2})

In [12]:
result2["answer"]

" I don't know."

In [13]:
query3 = "What is that document about then?"
result3 = qa({"question": query3})

In [14]:
result3["answer"]

' The document is a test data file.'

In [15]:
result3

{'question': 'What is that document about then?',
 'chat_history': [HumanMessage(content='From whom is the Document?'),
  AIMessage(content=' The document is from Linea Schmidt.'),
  HumanMessage(content='Is there any information about her profession?'),
  AIMessage(content=" I don't know."),
  HumanMessage(content='What is that document about then?'),
  AIMessage(content=' The document is a test data file.')],
 'answer': ' The document is a test data file.'}

## Query with Memory from Web

In [10]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [18]:
from langchain.memory import ConversationSummaryMemory
llm = OpenAI(temperature=0)
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

retriever = vectorstore.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [21]:
qa("How do agents use Task decomposition?")

{'question': 'How do agents use Task decomposition?',
 'chat_history': [SystemMessage(content='')],
 'answer': 'Agents utilize task decomposition by breaking down large tasks into smaller, manageable subgoals. This allows them to efficiently handle complex tasks. Task decomposition can be achieved through different methods such as using simple prompting, task-specific instructions, or human inputs. The agent can then plan ahead and determine the necessary steps to achieve each subgoal. This approach enables the agent to tackle complicated tasks in a more organized and systematic manner.'}

In [22]:
qa("What are the various ways to implement memory to support it?")

{'question': 'What are the various ways to implement memory to support it?',
 'chat_history': [SystemMessage(content='\nThe human asks how agents use Task decomposition. The AI explains that agents utilize task decomposition by breaking down large tasks into smaller, manageable subgoals, which can be achieved through different methods such as using simple prompting, task-specific instructions, or human inputs. This allows the agent to plan ahead and determine the necessary steps to achieve each subgoal, enabling them to tackle complicated tasks in a more organized and systematic manner.')],
 'answer': "The different methods to implement memory in order to support task decomposition are:\n\n1. Long Term Memory Management: This involves storing and organizing information in a long-term memory system. It allows the agent to retain knowledge of past tasks, subgoals, and their corresponding solutions. This memory can be accessed during task decomposition to retrieve relevant information and

## Query from Neo4J database 

In [16]:
from neo4j import GraphDatabase

class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self._driver = GraphDatabase.driver(uri, auth=(user, pwd))

    def close(self):
        self._driver.close()

In [17]:
class Neo4jLoader:
    def __init__(self, connection, query):
        self.connection = connection
        self.query = query

    def load(self):
        with self.connection._driver.session() as session:
            results = session.run(self.query)
            data = [record for record in results]
        return data

In [54]:
class Neo4jService(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def fetch_data(self):
        with self._driver.session() as session:
            return session.read_transaction(self._fetch_data_from_db)

    @staticmethod
    def _fetch_data_from_db(tx):
        query = """
        MATCH (s:Study)
        OPTIONAL MATCH (s)-[:HAS_INDICATION]->(ind:Indication)
        OPTIONAL MATCH (s)-[:HAS_SUBINDICATION]->(sub:SubIndication)
        OPTIONAL MATCH (s)-[relContact:HAS_CONTACT]->(con:Contact)
        OPTIONAL MATCH (s)-[:CONDUCTED_AT]->(sc:StudyCenter)
        OPTIONAL MATCH (s)-[:HAS_CRITERIA]->(criteria)
        WITH s, ind, sub, 
            COLLECT(DISTINCT con.name) AS ContactNames, 
            COLLECT(DISTINCT relContact.email) AS ContactEmails,
            COLLECT(DISTINCT sc.name) AS StudyCenterNames, 
            COLLECT(DISTINCT criteria.value) AS CriteriaValues
        RETURN 
        s.name AS StudyName,
        s.identifier AS Identifier,
        ind.name AS Indication,
        sub.name AS SubIndication,
        ContactNames,
        ContactEmails,
        StudyCenterNames,
        CriteriaValues,
        s.embedding AS Embedding,
        s.metadata AS Metadata

        """
        result = tx.run(query)
        return [{
            "StudyName": record["StudyName"],
            "Identifier": record["Identifier"],
            "Indication": record["Indication"],
            "SubIndication": record["SubIndication"],
            "ContactNames": ", ".join(record["ContactNames"]) if record["ContactNames"] else None,  # Joining list to string
            "ContactEmails": ", ".join(record["ContactEmails"]) if record["ContactEmails"] else None,  # Joining list to string
            "StudyCenterNames": ", ".join(record["StudyCenterNames"]) if record["StudyCenterNames"] else None,  # Joining list to string
            "CriteriaValues": ", ".join(record["CriteriaValues"]) if record["CriteriaValues"] else None,  # Joining list to string
            "Embedding": record["Embedding"],
            "Metadata": record["Metadata"],
            "SummaryText": " ".join([
                f"Study Name: {record['StudyName']}",
                f"Identifier: {record['Identifier']}",
                f"Indication: {record['Indication']}",
                f"Sub-Indication: {record['SubIndication']}",
                f"Contact Names: {', '.join(record['ContactNames']) if record['ContactNames'] else 'N/A'}",  # Joining list to string
                f"Contact Emails: {', '.join(record['ContactEmails']) if record['ContactEmails'] else 'N/A'}",  # Joining list to string
                f"Study Center Names: {', '.join(record['StudyCenterNames']) if record['StudyCenterNames'] else 'N/A'}",  # Joining list to string
                f"Criteria Values: {', '.join(record['CriteriaValues']) if record['CriteriaValues'] else 'N/A'}"  # Joining list to string
            ])
        } for record in result]



In [55]:
# Use your Neo4j credentials here
neo_service = Neo4jService(uri="bolt://localhost:7687", user="neo4j", password="password")
data = neo_service.fetch_data()
neo_service.close()

C:\Users\larsm\AppData\Local\Temp\ipykernel_29436\3977427155.py:11: DeprecationWarning: read_transaction has been renamed to execute_read
  return session.read_transaction(self._fetch_data_from_db)


In [56]:
data[2]

{'StudyName': "Relapsing Forms of Multiple Sclerosis (RMS) Study of Bruton's Tyrosine Kinase (BTK) Inhibitor Tolebrutinib (SAR442168) (GEMINI 1)",
 'Identifier': 'NCT04410978',
 'Indication': 'Multiple Sclerosis',
 'SubIndication': 'Relapsing Forms of Multiple Sclerosis (RMS)',
 'ContactNames': 'Marc Pawlitzki',
 'ContactEmails': 'marcguenter.pawlitzki@med.uni-duesseldorf.de',
 'StudyCenterNames': 'University Clinic Düsseldorf',
 'CriteriaValues': None,
 'Embedding': [-0.03976777433307709,
  -0.0020321362426919125,
  -0.009751431312557965,
  -0.02449853076228362,
  -0.014246121496091786,
  0.02633309833618682,
  -0.023665919095648157,
  -0.010090120725145056,
  -0.047585854552244726,
  -0.012108144870174068,
  0.006745563125093482,
  0.01735782936829017,
  0.011367261954762781,
  0.011600110285632159,
  -0.017922311412161142,
  0.007789855558180559,
  0.03138521579532866,
  -0.002492542061064087,
  -0.009306901749575701,
  -0.0014870579547576477,
  -0.003993712229899954,
  0.0018116352

In [61]:
json_data = json.dumps(data, indent=4)  # 'indent=4' is optional but makes the output more readable
json_data

'[\n    {\n        "StudyName": "Meteoroid Study",\n        "Identifier": "NCT05271409",\n        "Indication": "MOGAD",\n        "SubIndication": "Serum positivity for MOG-IgG by a CBA",\n        "ContactNames": " Orhan Aktas, Marc Pawlitzki",\n        "ContactEmails": "UNKNOWN_VALUE, marcguenter.pawlitzki@med.uni-duesseldorf.de",\n        "StudyCenterNames": ", University Clinic D\\u00fcsseldorf",\n        "CriteriaValues": null,\n        "Embedding": [\n            -0.0306129038136644,\n            -0.010464758056187703,\n            0.002522006488512908,\n            -0.029329228335423094,\n            -0.0333755996871782,\n            0.00934851668773799,\n            -0.014190211469705094,\n            -0.010157791330618055,\n            -0.02955247567779043,\n            -0.01228562507134524,\n            0.012048424653664621,\n            -0.0018138912924081773,\n            0.004349850739819043,\n            0.005759104827202515,\n            -0.02918969758229025,\n           

In [62]:
json_data = json.dumps(data, indent=4)  # 'indent=4' is optional but makes the output more readable

loader = JSONLoader(json_data, jq_schema= ".[].SummaryText",)
data = loader.load()


ImportError: jq package not found, please install it with `pip install jq`

In [41]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

retriever = vectorstore.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

response = qa("How do agents use Task decomposition?")
print(response)

AttributeError: 'NoneType' object has no attribute 'page_content'